# FINAL WORK 

## Authors:
**Full name: DIEGO HERNÁNDEZ SUÁREZ AND ÁNGELA DURÁN PINTO**
    
**NIA : 100472809 AND 100472766**

First, we need to import to import the required libraries

In [0]:
import pandas as pd
from pyspark.sql.functions import split, explode, collect_list, count, length
import numpy as np
import re
from pyspark.sql.types import *


Now, we read the Wikipedia dataset stored in Parquet format from the specified path. The dataset is loaded into a Spark DataFrame named wikipediaDF. This dataset contains information about Wikipedia articles. <br>

Then, we take a small random sample from the entire Wikipedia dataset. The fraction parameter determines the proportion of the dataset to be sampled. . The seed=123 ensures reproducibility by setting the random seed. Finally, the sampled DataFrame is cached for efficient reuse during subsequent operations. <br> 

In [0]:
wiki_data =spark.read.parquet("dbfs:/databricks-datasets/wikipedia-datasets/data-001/en_wikipedia/articles-only-parquet")

# Obtain a sample from the whole data set.
sample_data = wiki_data.sample(fraction=0.0001, seed=123).cache()


Now, we define the function **"document_links_extractor"** to extract links from the content of a document. The function uses the re.findall method to identify substrings enclosed within double square brackets, representing potential links. If links are found, they are converted to lowercase, and the resulting list is returned. In the absence of links, an empty list is initialized. This function is intended to be used as part of a larger process, such as parsing and processing text data, especially in scenarios where links within documents need to be extracted and processed. The conversion to lowercase ensures uniformity and consistency in handling links. <br>

Then, we creates a User-Defined Function (UDF) named **"document_links_extractor"** using the previously defined parse_links function. This UDF is designed to extract links from document content and is configured to return an array of strings. The udf function allows this functionality to be applied efficiently across Spark DataFrames, enabling distributed and parallelized processing. Consequently, when incorporated into Spark DataFrame operations, parse_links_udf can be employed to extract and manipulate link information within a distributed computing environment, in this case, a Spark cluster.

In [0]:
# Define a function to parse links from document body
def document_links_extractor(content):
    data = re.findall(r'\[\[(.+?)\]\]', content)
    
    # If links are found, convert them to lowercase; otherwise, initialize an empty list
    links = [z.lower() for z in data] if len(data) > 0 else []

    return links

# Create a UDF for link extraction
extract_links_udf = udf(document_links_extractor, ArrayType(StringType()))


We define a UDF using a lambda function to convert strings to lowercase

In [0]:
to_lower_udf = udf(lambda x: x.lower())

Now, we apply the previous defined functions to obtain our sample parsed saving the extracted links.

In [0]:
# Apply UDFs to select and transform columns
parsed_sample_data = sample_data.select(
    to_lower_udf("title").alias("title"),
    "id",
    extract_links_udf("text").alias("text_links")
).cache()

We want to save all the titles(in lower case) with their corresponding ids from the entire wikipedia dataframe. <br>

We save it as a broadcast variable. This is a strategy in Spark to efficiently share read-only data across tasks in a distributed computation. It helps in avoiding unnecessary data shuffling and reduces the overhead of sending large datasets to each worker node independently. 

In [0]:

# Compute the panda DF obtaining the id of the titles
titles_id_pandas_df = wiki_data.select(to_lower_udf("title").alias("title"), "id").toPandas()

# Compute the broadcoast to the titles id panda DF
titles_id_BC = sc.broadcast(titles_id_pandas_df)


**convert_titles_to_id** is designed to map a list of titles to their corresponding IDs. The function takes two parameters: titles, representing the input list of titles, and titles_data_broadcast, which is a broadcasted Pandas DataFrame containing information about titles and their associated IDs. Within the function, two lists are initialized: title_id_list to store the resulting IDs and titles_data to access the broadcasted data efficiently. The function then iterates over the input titles, searching for matching rows in the broadcasted titles data. If matches are found, the ID value is extracted from the last occurrence in the row and appended to the title_id_list. The function ultimately returns a list of integer IDs corresponding to the input list of titles. This function is particularly useful in scenarios where efficient title-to-ID mapping is required, leveraging the broadcasted data to avoid redundant transfers in a distributed computing environment.

Again, we define its corresponding udf.

In [0]:
def convert_titles_to_id(titles, titles_data_broadcast):
   
    title_id_list, titles_data = [], titles_data_broadcast.value

    # Iterate over input titles
    for title in titles:
        # Find the row in titles_data corresponding to the title
        matching_rows = titles_data[titles_data["title"] == title]

        # Check if there are matching rows
        if not matching_rows.empty:
            # Extract the ID value from the last occurrence in the row
            id_value = int(matching_rows["id"].iloc[-1])
            title_id_list.append(id_value)

    return title_id_list

convert_titles_to_id_udf = udf(lambda titles: convert_titles_to_id(titles, titles_id_BC), ArrayType(LongType()))

### Forward Link Martix
Create a new DataFrame by applying the convert_titles_to_id_udf User-Defined Function (UDF) to the "text_links" column of the existing DataFrame ParsedWikipediaDF. This transformation effectively maps titles to corresponding IDs, and the resulting DataFrame contains two columns: "id" and "links."

In [0]:
auxForward_df = parsed_sample_data.select(
    "id",
    convert_titles_to_id_udf("text_links").alias("links")
).cache()

Now, we define a simple Python function that takes a list of links as input and returns the count of elements in the list using the len function. Subsequently, a User-Defined Function (UDF) is created using wrapping the functionality of the counting function. This UDF is configured to return a long integer (LongType()). The purpose of this UDF is to help us to calculate the number of outlinks associated with each record.

In [0]:
def count_links(links):
    return len(links)

count_links_udf = udf(count_links, LongType())

Finally, we create our last version of our **Forward Link Martix** by selecting the "id" and "links" columns from the existing DataFrame auxForward_df. Additionally, it applies the previously defined User-Defined Function (UDF) named count_links_udf to the "links" column, creating a new column named "outgoing" that represents the count of outgoing links associated with each record.

In [0]:
forward_df = auxForward_df.select(
    "id",
    "links",
    count_links_udf("links").alias("outgoing")
).cache()
forward_df.toPandas()

### Backward Link Matrix
Using the forward_df DataFrame, the backward_df DataFrame is computed through a multi-step process. Initially, the ForwardLinks DataFrame is expanded, resulting in a comprehensive DataFrame with "id" representing the source document's ID, "target" indicating the target document's ID, and "outgoing" denoting the count of outgoing links from the source document. <br>

Subsequently, the DataFrame is grouped by the "target" column, leading to the creation of the backward_df DataFrame. This new DataFrame encompasses columns like "id" for the target document's ID, "links" presenting a list of IDs for all documents pointing to the target document, and "outgoing" containing a list of counters representing the outgoing links for each document ID in the "links" list. It's noteworthy that the backward_df DataFrame is strategically cached for subsequent PageRank calculations. Additionally, last nodes are stored separately to ensure their inclusion in the final output, given that they may not be represented explicitly in the backward_df.

In [0]:
aux_backward_df = forward_df.select(
    "id",
    explode("links").alias("target"),
    "outgoing"
).distinct()


backward_df = aux_backward_df.groupBy("target").agg(
    collect_list("id").alias("links"),
    collect_list("outgoing").alias("outgoing")
).withColumnRenamed("target", "id").cache()

## Page Rank Calculation
 

### Initialization
In this code segment, a Pandas DataFrame named pagerank_pandas_df is created by selecting the "id" column from the Spark DataFrame backward_df and converting it to Pandas using the toPandas() method. Subsequently, two new columns are added to this Pandas DataFrame: "PGRK" and "UpdatedPGRK." The "PGRK" column is initialized with a calculated value, where each row is assigned 0.85 divided by the total count of records in the backward_df. The "UpdatedPGRK" column is set equal to the "PGRK" column initially.

In [0]:
pagerank_pandas_df = backward_df.select("id").toPandas()

pagerank_pandas_df["PGRK"] = (0.85 / backward_df.count())
pagerank_pandas_df["UpdatedPGRK"] = pagerank_pandas_df["PGRK"]


### Calculation

**calculate_page_rank_iteration**, is used in an iterative process to compute the PageRank values for a set of documents based on incoming links. <br> 

The function takes four main parameters:
- links representing the incoming links' IDs
- counters indicating the count of outgoing links for each incoming link
- current_pgrk denoting the current PageRank value
- pagerank_data containing the existing PageRank values for all documents
- dp representing the damping factor.

Within the function, a new PageRank value (new_pgrk) is initialized with the current PageRank value. It then iterates over the incoming links and corresponding counters. For each link, it retrieves the matching row from the pagerank_data DataFrame based on the link's ID. If the row is not empty and the counter is positive (to avoid division by zero), it updates the new_pgrk using the PageRank formula specified in the code. The function ultimately returns the updated PageRank value.

The use of the damping factor (dp) ensures a balance between existing PageRank values and the influence of incoming links in the iterative update process.

In [0]:
def calculate_page_rank_iteration(links, counters, current_pgrk, pagerank_data, dp):
    
    new_pgrk = current_pgrk

    # Iterate over incoming links
    for link_id, counter in zip(links, counters):
        # Find the row in pagerank_data corresponding to the incoming link
        matching_row = pagerank_data[pagerank_data['id'] == link_id]

        # Check if the row is not empty
        if not matching_row.empty:
          # check the counter is positive to avoid division by zero
          if counter > 0:
            link_pgrk_value = matching_row.iloc[0]["PGRK"]

            # Update the PageRank
            new_pgrk = new_pgrk + (1 - dp) + dp * (link_pgrk_value / counter)
    # Return the new pagerank
    return new_pgrk

Then, we define an iterative process for calculating PageRank values using the PageRank algorithm with a maximum of 20 iterations and a specified tolerance level. Within each iteration, a loop iterates over nodes in the backward_df DataFrame to calculate new PageRank values using the calculate_page_rank_iteration function. The results are stored in a list named new_pgrk_values. Subsequently, the DataFrame columns storing the previous and new PageRank values ('PR' and 'UpdatedPGRK') are updated based on the calculated values.

A convergence check is performed by comparing the absolute difference between the new and previous PageRank values relative to the previous values. If the convergence condition is satisfied (i.e., the difference is lower or equal to the established tolerance), the iteration process is halted, and a message indicating convergence is printed. This convergence check ensures that the iterative algorithm stops when the PageRank values stabilize within the defined tolerance, optimizing computational resources. Overall, the code orchestrates the iterative computation of PageRank values while monitoring convergence.

In [0]:

max_iterations = 20
tolerance = 0.0001
for iteration in range(max_iterations):
    print(f"Iteration {iteration+1}")
    new_pgrk_values = []
    
    # Iterate over nodes to calculate new PageRank values
    for row_data in backward_df.collect():
      # Append to the new page rank by using the calculate page rank function.
        new_pgrk_values.append(calculate_page_rank_iteration(row_data[1], row_data[2], pagerank_pandas_df[pagerank_pandas_df['id'] == row_data[0]].iloc[0]["PGRK"], pagerank_pandas_df, 0.85))

    # Update PageRank values in the DataFrame
    pagerank_pandas_df['UpdatedPGRK'], previous_pgrk, pagerank_pandas_df['PGRK'] = new_pgrk_values, pagerank_pandas_df['PGRK'], pagerank_pandas_df['UpdatedPGRK']
    
    # Check convergence in the case the difference between the page ranks is lower or equal to the tolerance established
    if (abs((pagerank_pandas_df['UpdatedPGRK'] - previous_pgrk) / previous_pgrk) <= tolerance).head()[0]:
      print("Convergence condition satisfied. Stoping loop")
      break



Now, we define a Pandas DataFrame named lastnode by filtering the forward_df DataFrame to select records where the "outgoing" column is equal to 0, indicating final nodes with no outgoing links. The resulting DataFrame consists of the "id" column representing the IDs of these last nodes. Subsequently, the "PGRK" column in the lastnode DataFrame is initialized with PageRank values calculated as 0.85 divided by the total count of records in the original forward_df. Finally, the resulting DataFrame is displayed. This code snippet is likely addressing the handling of final nodes in the PageRank algorithm, initializing their PageRank values before the final output.

In [0]:
lastnode = forward_df.filter(forward_df["outgoing"] == 0).select("id").toPandas()

# Calculate PageRank for final nodes
lastnode["PGRK"] = 0.85 / forward_df.count()

# Display the resulting DataFrame
display(lastnode)

We select only the "id" and "PGRK" columns from the existing DataFrame page_rank_pdf. The code then concatenates this DataFrame with another DataFrame named lastnode, representing final nodes and their corresponding PageRank values. The pd.concat function is utilized to combine the two DataFrames, ensuring a continuous index in the resulting pagerank_pdf DataFrame. Finally, the combined DataFrame is displayed, providing a view of the calculated PageRank values for all nodes, including last nodes. This operation is crucial for consolidating PageRank information and presenting the final results in a comprehensive format.

In [0]:
# Select 'id' and 'PR' columns from PageRankPDF
pagerank_pdf = pagerank_pandas_df[['id', 'PGRK']]

# Concatenate PageRankPDF and final
pagerank_pdf = pd.concat([pagerank_pandas_df, lastnode])

# Display the resulting DataFrame
display(pagerank_pdf)

## Final Results

Construct the final results by merging two existing DataFrames: pagerank_pandas_df and a subset of the titles_id_pandas_df DataFrame containing only the "id" and "title" columns. The merge operation is performed based on the "id" column, linking the PageRank values with their corresponding titles. Following the merge, duplicates in the resulting DataFrame are removed, specifically based on the "id" column, using the drop_duplicates method. This ensures that each unique ID is associated with only one set of PageRank and title values. Finally, the cleaned and merged DataFrame is displayed, offering a comprehensive view of the final PageRank results with associated titles for each document in the dataset.

In [0]:
# Merge pagerank_pandas_df with titles_id_pandas_df to get titles for corresponding IDs
final_results = pd.merge(pagerank_pandas_df, titles_id_pandas_df[["id", "title"]])

# Remove duplicates based on 'id' column
final_results = final_results.drop_duplicates(subset=['id'])

# Display the resulting DataFrame
display(final_results)

We can see that we obtain appropiate results, the contributions are well computed. If we put just a few data, the results are going to be the same as maybe the pages does not have references.

# CONCLUSIONS


Nowadays, the huge amount of data can be difficult to deal with, one example could be articles in Wikipedia, in which there exist a huge amount of articles that are interconnected between them, creating a network of different articles referenced between them, for this reason we will analyze and see the relationships between them.

During this assigment we faced several challenges that we solved by optimizing or trying different approaches with the objective of obtaining results eventhough the code faces a huge data set. 
Our initial steps was to load a significantly big data set obtained from Wikipedia articles into a PySpark DataFrame in order to manipulate it. 
Due to computational limitations, we were forced to take a small sample of the data for subsequent analysis and manipulation. 
Our main objective was to understand and work with the network structure of the Wikipedia articles, were our special focus was with the linkages between the different articles and its relationships. However, in order to achieve this task it was necessary to have an optimized code to be able to analyze and process bigger samples of the whole dataset.

Our next step was to extract the links from the articles, the link extraction process was essential in order to understand  the relationships and be able to work properly with it, it is for this reason that this first steps were essential in order to obtain reliable and significant results. 
In our case, thanks to a UDF (user-defined function) that were made in order to parse links from the content of these articles, after that this function is used in order to create a new DataFrame which contains the articles IDs, titles and the extracted links. 

In order to amange the mapping between the article IDs and their respective titles, we created a PD Dataframe from the original wikipedia and to take into consideration potential situations were the title is not included in our sample. 
We also broadcasted this data frame with optimization perspectives in order to achieve the fastest title to IDs transformations in PySpark.

The code proceeds with the developing of the forward and backward linkages between the different articles. Starting with the forward link data frame, it contains the information about the linked articles, which includes their IDs or also the count of outgoing links from them. 
Meanwhile, the backward data frame was created by adding linking article IDs with their corresponding outgoing link counts for each target article.

Now we reach the most important part of the assigment, which is the computation of the pagerank respective value, using an algorithm which ranks articles based on their importance through a network. By a iterative process where we placed a maximum of 20 iterations in order to avoid infinite iterations or until it converges, where we used a treshold which was the tolerance in order to accept that the algorithm already converged, this was done by a condition when substracting the updated pagerank with the old one and dividing this substraction by the old page rank. 

After this process finishes, we can display and analyze the pageranks values obtained for the different articles, where in the final results we can see the pageranks values for each article in the sample within the network. We decided to remove duplicates in order to enhance computational efficiency.


If we look this tables, we can see that each row corresponds to a unique article ID and the columns computes the pagerank values (PGRK and UpdatedPGRK) and its ID and title. The data frame shows the relative importance of each article in the network, determined by the pagerank algorithm. The articles with the highest pagerank values normally implies a higher importance or centrality within the network. This assigment could be useful for users in order to identify key articles and gain knowledge related to how the network of Wikipedia works, based on linkages and connections between articles.

It is important to mention that we tried different codes in order to analyze and decide which was the most optimal, we even tried with SQL queries in order to judge if it could be a potential solution, however, this code was the most efficient in terms of time and also shows the different content learned during the course.

To sum up, the computed assigment shows a systematic exploration through the wikipedia articles in order to understand the network it composes and the relationship between the different articles by computing the pageranks of each article. The results obtained shows interesting insights related to the importance of each article analyzed. However, we must mention the computational cost that this type of codes supposes, where in our case, we tried to optimize and be able to compute the pageranks despite using a lot of data, but we consider we could optimize even more this code by avoiding the unnecesary use of panda data frames conversions, eventhough this detail, we consider that its good to learn from this type of mistakes in order to improve. During the whole assigment we employed several aspects that we studied during the course of Massive Computing, in this way we were able to manage huge data sets and work with them in order to manipulate them and obtain significant insights from it.

